In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold,RandomizedSearchCV
from tqdm import tqdm
from utils import LoadRumoursDataset,Load_Rumours_Dataset_filtering_since_first_post
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pylab as plt
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import uuid

In [2]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"


### SMOTE APPROACH

In [3]:
dataset_generator  = Load_Rumours_Dataset_filtering_since_first_post(file_path_replies, file_path_posts,time_cut=24*3*60)
dataset_generator.load_data()
dataset_generator.process_data()
train_dataset, test_dataset = dataset_generator.get_final_dataframes()

In [4]:
X_train  = train_dataset.drop(columns=['rumour'])
X_train = np.hstack([X_train.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))])


X_test  = test_dataset.drop(columns=['rumour'])
X_test = np.hstack([X_test.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))])

#X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
y_train =train_dataset['rumour']
y_test =test_dataset['rumour']

In [ ]:
print(y_train.sum(),y_train.shape)

In [ ]:
print(y_test.sum(),y_test.shape)

In [5]:
# Apply SMOTE to the features without embeddings
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42,sampling_strategy='minority')
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [6]:

# Define the parameter grid to search
param_dist = {
    'n_estimators': [100, 200, 400, 500],  # Number of trees
    'max_depth': [None, 10, 20, 30, 40, 50],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at a leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider for the best split
    'bootstrap': [True, False],  # Whether to bootstrap samples when building trees
}

# Initialize the RandomForestClassifier
rf_model = RandomForestClassifier()

# Use RandomizedSearchCV to search the parameter space
random_search_rf = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=param_dist,
    n_iter=10,  # Number of different combinations to try
    scoring='recall',  # F1 score for imbalanced classification
    cv=5,  # 5-fold cross-validation
    verbose=2,
    n_jobs=-1,
    random_state=42
)



In [7]:
# Fit the random search model
random_search_rf.fit(X_resampled, y_resampled)

# Print the best parameters and the corresponding score
print("Best parameters found: ", random_search_rf.best_params_)
print("Best Recall: ", random_search_rf.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters found:  {'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'log2', 'max_depth': 40, 'bootstrap': False}
Best Recall:  0.9494778675009512


In [9]:
best_params =  random_search_rf.best_params_

In [48]:
def modelfit(alg, X_train, y_train, X_test, y_test, printFeatureImportance=True):
    with mlflow.start_run():
        # Fit the algorithm on the data
        #alg.fit(X_resampled, y_resampled)
            
        # Predict training set:
        dtrain_predictions = alg.predict(X_train)
        dtrain_predprob = alg.predict_proba(X_train)[:, 1]

        # Predict test set:
        dtest_predictions = alg.predict(X_test)
        dtest_predprob = alg.predict_proba(X_test)[:, 1]

        # Log model parameters
        mlflow.log_params(alg.get_params())

        # Log metrics for Train
        mlflow.log_metric("train_auc", roc_auc_score(y_train, dtrain_predprob))
        mlflow.log_metric("train_accuracy", accuracy_score(y_train, dtrain_predictions))
        mlflow.log_metric("train_precision", precision_score(y_train, dtrain_predictions))
        mlflow.log_metric("train_recall", recall_score(y_train, dtrain_predictions))

        # Log metrics for Test
        mlflow.log_metric("test_auc", roc_auc_score(y_test, dtest_predprob))
        mlflow.log_metric("test_accuracy", accuracy_score(y_test, dtest_predictions))
        mlflow.log_metric("test_precision", precision_score(y_test, dtest_predictions))
        mlflow.log_metric("test_recall", recall_score(y_test, dtest_predictions))

        # Print model report for Train
        print("\nModel Report Train")
        print("AUC Score : %f" % roc_auc_score(y_train, dtrain_predprob))
        print("Accuracy : %.4g" % accuracy_score(y_train, dtrain_predictions))
        print("Precision : %.4g" % precision_score(y_train, dtrain_predictions))
        print("Recall : %f" % recall_score(y_train, dtrain_predictions))

      

        # Log confusion matrix for Train
        #train_cm = confusion_matrix(y_train, dtrain_predictions)
        #mlflow.log_artifact(plot_confusion_matrix(train_cm, "Train"))

        # Print model report for Test
        print("\nModel Report Test")
        print("AUC Score : %f" % roc_auc_score(y_test, dtest_predprob))
        print("Accuracy : %.4g" % accuracy_score(y_test, dtest_predictions))
        print("Precision : %.4g" % precision_score(y_test, dtest_predictions))
        print("Recall : %f" % recall_score(y_test, dtest_predictions))


        # Log the model with input example
        #input_example = X_train[0:1]   # Use the first row of X_train as an example
        #mlflow.sklearn.log_model(alg, "model", input_example=input_example)
        mlflow.log_metric("time_cut", time_cut)
    
    # Explicitly end the MLflow run
    mlflow.end_run()

    return alg



In [38]:
# Train the final model with the best hyperparameters
best_params = {'n_estimators': 150, 'min_samples_split': 10, 'min_samples_leaf': 5, \
               'max_features': 'sqrt', 'max_depth': 5, 'bootstrap': True, 'class_weight': 'balanced'}
final_model = RandomForestClassifier(
    **best_params
)
final_model.fit(X_resampled, y_resampled)
    

RandomForestClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=5,
                       min_samples_split=10, n_estimators=150)

In [39]:
modelfit(final_model,X_resampled, y_resampled,X_test,y_test)


Model Report Train
AUC Score : 0.974704
Accuracy : 0.9044
Precision : 0.8833
Recall : 0.931985

Model Report Test
AUC Score : 0.903353
Accuracy : 0.8136
Precision : 0.5539
Recall : 0.843284


<Figure size 640x480 with 0 Axes>

CV Score : Mean - 0.8970659 | Std - 0.02821971 | Min - 0.8577982 | Max - 0.9357798


RandomForestClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=5,
                       min_samples_split=10, n_estimators=150)

In [40]:
modelfit(final_model,X_train, y_train,X_test,y_test)


Model Report Train
AUC Score : 0.963799
Accuracy : 0.8837
Precision : 0.6794
Recall : 0.907348

Model Report Test
AUC Score : 0.903353
Accuracy : 0.8136
Precision : 0.5539
Recall : 0.843284


<Figure size 640x480 with 0 Axes>

CV Score : Mean - 0.7603175 | Std - 0.04454167 | Min - 0.6935484 | Max - 0.8095238


RandomForestClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=5,
                       min_samples_split=10, n_estimators=150)

In [19]:
X_train.shape

(1401, 108)

#### Without SMOTE

In [44]:

final_model.fit(X_train, y_train)
    

RandomForestClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=5,
                       min_samples_split=10, n_estimators=150)

In [45]:
modelfit(final_model, X_train, y_train,X_test,y_test)


Model Report Train
AUC Score : 0.969282
Accuracy : 0.8965
Precision : 0.7029
Recall : 0.929712

Model Report Test
AUC Score : 0.900780
Accuracy : 0.827
Precision : 0.5798
Recall : 0.813433


<Figure size 640x480 with 0 Axes>

CV Score : Mean - 0.7635433 | Std - 0.04381894 | Min - 0.6935484 | Max - 0.8225806


RandomForestClassifier(class_weight='balanced', max_depth=5, min_samples_leaf=5,
                       min_samples_split=10, n_estimators=150)

In [46]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("Random Forest SMOTE Filter Node on Test  2024-10-30 10 min-3days")

2024/10/31 00:38:00 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest SMOTE Filter Node on Test  2024-10-30 10 min-3days' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/azureuser/rumour-detection-pheme/mlruns/44', creation_time=1730335080083, experiment_id='44', last_update_time=1730335080083, lifecycle_stage='active', name='Random Forest SMOTE Filter Node on Test  2024-10-30 10 min-3days', tags={}>

In [49]:
#
file_path_replies = r"/home/azureuser/rumour-detection-pheme/replies_charlie_hebdo.pkl"
file_path_posts = r"/home/azureuser/rumour-detection-pheme/posts_charlie_hebdo.pkl"

for time_cut in range(14,24*3*60,10):
    print(time_cut)
    dataset_generator  = Load_Rumours_Dataset_filtering_since_first_post(file_path_replies, file_path_posts,time_cut=24*3*60)
    dataset_generator.load_data()
    dataset_generator.process_data()
    train_dataset, test_dataset = dataset_generator.get_final_dataframes()

    X_train  = train_dataset.drop(columns=['rumour'])
    X_train = np.hstack([X_train.drop(columns=['embeddings_avg']).values, \
                             np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))])


    X_test  = test_dataset.drop(columns=['rumour'])
    X_test = np.hstack([X_test.drop(columns=['embeddings_avg']).values, \
                            np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))])

    #X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
    y_train =train_dataset['rumour']
    y_test =test_dataset['rumour']
        
    #if time_cut_posts >20:

    smote = SMOTE(random_state=42,sampling_strategy='minority')
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    final_model = RandomForestClassifier(
    **best_params)
        
    final_model.fit(X_train, y_train)
         
        
    modelfit(final_model,  X_train, y_train,X_test,y_test)

14

Model Report Train
AUC Score : 0.967661
Accuracy : 0.8929
Precision : 0.6983
Recall : 0.916933

Model Report Test
AUC Score : 0.899070
Accuracy : 0.8236
Precision : 0.5745
Recall : 0.805970
24

Model Report Train
AUC Score : 0.969108
Accuracy : 0.8944
Precision : 0.6978
Recall : 0.929712

Model Report Test
AUC Score : 0.899565
Accuracy : 0.8153
Precision : 0.5596
Recall : 0.805970
34

Model Report Train
AUC Score : 0.967138
Accuracy : 0.8944
Precision : 0.6978
Recall : 0.929712

Model Report Test
AUC Score : 0.899949
Accuracy : 0.817
Precision : 0.5632
Recall : 0.798507
44

Model Report Train
AUC Score : 0.969657
Accuracy : 0.8951
Precision : 0.7015
Recall : 0.923323

Model Report Test
AUC Score : 0.899469
Accuracy : 0.822
Precision : 0.5707
Recall : 0.813433
54

Model Report Train
AUC Score : 0.967041
Accuracy : 0.8929
Precision : 0.6964
Recall : 0.923323

Model Report Test
AUC Score : 0.900876
Accuracy : 0.827
Precision : 0.5815
Recall : 0.798507
64

Model Report Train
AUC Score 


Model Report Train
AUC Score : 0.968568
Accuracy : 0.8901
Precision : 0.6934
Recall : 0.910543

Model Report Test
AUC Score : 0.897456
Accuracy : 0.8203
Precision : 0.5691
Recall : 0.798507
454

Model Report Train
AUC Score : 0.967743
Accuracy : 0.8936
Precision : 0.6971
Recall : 0.926518

Model Report Test
AUC Score : 0.899134
Accuracy : 0.8153
Precision : 0.5602
Recall : 0.798507
464

Model Report Train
AUC Score : 0.967499
Accuracy : 0.8944
Precision : 0.696
Recall : 0.936102

Model Report Test
AUC Score : 0.896961
Accuracy : 0.812
Precision : 0.5538
Recall : 0.805970
474

Model Report Train
AUC Score : 0.967020
Accuracy : 0.8915
Precision : 0.6968
Recall : 0.910543

Model Report Test
AUC Score : 0.901147
Accuracy : 0.822
Precision : 0.5707
Recall : 0.813433
484

Model Report Train
AUC Score : 0.967505
Accuracy : 0.8894
Precision : 0.6881
Recall : 0.923323

Model Report Test
AUC Score : 0.897887
Accuracy : 0.8103
Precision : 0.5521
Recall : 0.791045
494

Model Report Train
AUC Scor


Model Report Train
AUC Score : 0.966313
Accuracy : 0.8908
Precision : 0.6942
Recall : 0.913738

Model Report Test
AUC Score : 0.895650
Accuracy : 0.822
Precision : 0.5707
Recall : 0.813433
884

Model Report Train
AUC Score : 0.968283
Accuracy : 0.8865
Precision : 0.6851
Recall : 0.910543

Model Report Test
AUC Score : 0.897216
Accuracy : 0.8103
Precision : 0.5521
Recall : 0.791045
894

Model Report Train
AUC Score : 0.968591
Accuracy : 0.8908
Precision : 0.6923
Recall : 0.920128

Model Report Test
AUC Score : 0.903337
Accuracy : 0.827
Precision : 0.5781
Recall : 0.828358
904

Model Report Train
AUC Score : 0.967100
Accuracy : 0.8922
Precision : 0.6947
Recall : 0.923323

Model Report Test
AUC Score : 0.898447
Accuracy : 0.8203
Precision : 0.5684
Recall : 0.805970
914

Model Report Train
AUC Score : 0.968054
Accuracy : 0.8894
Precision : 0.6908
Recall : 0.913738

Model Report Test
AUC Score : 0.898255
Accuracy : 0.8186
Precision : 0.5648
Recall : 0.813433
924

Model Report Train
AUC Sco


Model Report Train
AUC Score : 0.966504
Accuracy : 0.8901
Precision : 0.6906
Recall : 0.920128

Model Report Test
AUC Score : 0.898495
Accuracy : 0.8136
Precision : 0.5567
Recall : 0.805970
1304

Model Report Train
AUC Score : 0.968970
Accuracy : 0.8915
Precision : 0.6912
Recall : 0.929712

Model Report Test
AUC Score : 0.898894
Accuracy : 0.8253
Precision : 0.5775
Recall : 0.805970
1314

Model Report Train
AUC Score : 0.966912
Accuracy : 0.8936
Precision : 0.6981
Recall : 0.923323

Model Report Test
AUC Score : 0.901403
Accuracy : 0.8153
Precision : 0.5602
Recall : 0.798507
1324

Model Report Train
AUC Score : 0.967728
Accuracy : 0.8922
Precision : 0.6976
Recall : 0.913738

Model Report Test
AUC Score : 0.902170
Accuracy : 0.8286
Precision : 0.582
Recall : 0.820896
1334

Model Report Train
AUC Score : 0.966806
Accuracy : 0.8944
Precision : 0.6988
Recall : 0.926518

Model Report Test
AUC Score : 0.900604
Accuracy : 0.8253
Precision : 0.5767
Recall : 0.813433
1344

Model Report Train
A


Model Report Train
AUC Score : 0.967229
Accuracy : 0.8979
Precision : 0.7073
Recall : 0.926518

Model Report Test
AUC Score : 0.897025
Accuracy : 0.8186
Precision : 0.5668
Recall : 0.791045
1724

Model Report Train
AUC Score : 0.967455
Accuracy : 0.8929
Precision : 0.6973
Recall : 0.920128

Model Report Test
AUC Score : 0.899054
Accuracy : 0.817
Precision : 0.5632
Recall : 0.798507
1734

Model Report Train
AUC Score : 0.968377
Accuracy : 0.8908
Precision : 0.6951
Recall : 0.910543

Model Report Test
AUC Score : 0.899469
Accuracy : 0.8253
Precision : 0.5767
Recall : 0.813433
1744

Model Report Train
AUC Score : 0.968031
Accuracy : 0.8929
Precision : 0.6945
Recall : 0.929712

Model Report Test
AUC Score : 0.898974
Accuracy : 0.827
Precision : 0.5781
Recall : 0.828358
1754

Model Report Train
AUC Score : 0.967658
Accuracy : 0.8951
Precision : 0.7044
Recall : 0.913738

Model Report Test
AUC Score : 0.896817
Accuracy : 0.822
Precision : 0.5722
Recall : 0.798507
1764

Model Report Train
AUC

...


Model Report Train
AUC Score : 0.967496
Accuracy : 0.8922
Precision : 0.6966
Recall : 0.916933

Model Report Test
AUC Score : 0.897456
Accuracy : 0.8186
Precision : 0.5676
Recall : 0.783582
2564

Model Report Train
AUC Score : 0.969367
Accuracy : 0.8929
Precision : 0.6983
Recall : 0.916933

Model Report Test
AUC Score : 0.898670
Accuracy : 0.822
Precision : 0.5722
Recall : 0.798507
2574

Model Report Train
AUC Score : 0.967570
Accuracy : 0.8929
Precision : 0.6964
Recall : 0.923323

Model Report Test
AUC Score : 0.896401
Accuracy : 0.827
Precision : 0.5806
Recall : 0.805970
2584

Model Report Train
AUC Score : 0.965902
Accuracy : 0.8951
Precision : 0.7024
Recall : 0.920128

Model Report Test
AUC Score : 0.895219
Accuracy : 0.817
Precision : 0.5625
Recall : 0.805970
2594

Model Report Train
AUC Score : 0.966703
Accuracy : 0.8951
Precision : 0.7054
Recall : 0.910543

Model Report Test
AUC Score : 0.899326
Accuracy : 0.8136
Precision : 0.5567
Recall : 0.805970
2604

Model Report Train
AUC


Model Report Train
AUC Score : 0.967575
Accuracy : 0.8922
Precision : 0.6957
Recall : 0.920128

Model Report Test
AUC Score : 0.897025
Accuracy : 0.822
Precision : 0.5692
Recall : 0.828358
2984

Model Report Train
AUC Score : 0.968036
Accuracy : 0.8908
Precision : 0.6942
Recall : 0.913738

Model Report Test
AUC Score : 0.899693
Accuracy : 0.817
Precision : 0.5632
Recall : 0.798507
2994

Model Report Train
AUC Score : 0.967875
Accuracy : 0.8901
Precision : 0.6888
Recall : 0.926518

Model Report Test
AUC Score : 0.899821
Accuracy : 0.8186
Precision : 0.5641
Recall : 0.820896
3004

Model Report Train
AUC Score : 0.968183
Accuracy : 0.8936
Precision : 0.701
Recall : 0.913738

Model Report Test
AUC Score : 0.898958
Accuracy : 0.8253
Precision : 0.5784
Recall : 0.798507
3014

Model Report Train
AUC Score : 0.968019
Accuracy : 0.8929
Precision : 0.6964
Recall : 0.923323

Model Report Test
AUC Score : 0.899358
Accuracy : 0.8186
Precision : 0.5654
Recall : 0.805970
3024

Model Report Train
AUC


Model Report Train
AUC Score : 0.968310
Accuracy : 0.8922
Precision : 0.6957
Recall : 0.920128

Model Report Test
AUC Score : 0.899549
Accuracy : 0.8286
Precision : 0.5829
Recall : 0.813433
3404

Model Report Train
AUC Score : 0.967258
Accuracy : 0.8944
Precision : 0.6978
Recall : 0.929712

Model Report Test
AUC Score : 0.899549
Accuracy : 0.8203
Precision : 0.5684
Recall : 0.805970
3414

Model Report Train
AUC Score : 0.968389
Accuracy : 0.8951
Precision : 0.7015
Recall : 0.923323

Model Report Test
AUC Score : 0.899805
Accuracy : 0.8286
Precision : 0.5847
Recall : 0.798507
3424

Model Report Train
AUC Score : 0.968832
Accuracy : 0.8951
Precision : 0.7034
Recall : 0.916933

Model Report Test
AUC Score : 0.901643
Accuracy : 0.822
Precision : 0.5714
Recall : 0.805970
3434

Model Report Train
AUC Score : 0.966956
Accuracy : 0.8958
Precision : 0.7032
Recall : 0.923323

Model Report Test
AUC Score : 0.898814
Accuracy : 0.8153
Precision : 0.5596
Recall : 0.805970
3444

Model Report Train
A


Model Report Train
AUC Score : 0.966501
Accuracy : 0.8922
Precision : 0.6957
Recall : 0.920128

Model Report Test
AUC Score : 0.900316
Accuracy : 0.8236
Precision : 0.5745
Recall : 0.805970
3824

Model Report Train
AUC Score : 0.967878
Accuracy : 0.8965
Precision : 0.7059
Recall : 0.920128

Model Report Test
AUC Score : 0.900157
Accuracy : 0.8236
Precision : 0.5737
Recall : 0.813433
3834

Model Report Train
AUC Score : 0.968210
Accuracy : 0.8915
Precision : 0.6949
Recall : 0.916933

Model Report Test
AUC Score : 0.898431
Accuracy : 0.8253
Precision : 0.5784
Recall : 0.798507
3844

Model Report Train
AUC Score : 0.967558
Accuracy : 0.8929
Precision : 0.6983
Recall : 0.916933

Model Report Test
AUC Score : 0.900428
Accuracy : 0.822
Precision : 0.5722
Recall : 0.798507
3854

Model Report Train
AUC Score : 0.968213
Accuracy : 0.8944
Precision : 0.6988
Recall : 0.926518

Model Report Test
AUC Score : 0.898207
Accuracy : 0.8186
Precision : 0.5661
Recall : 0.798507
3864

Model Report Train
A


Model Report Train
AUC Score : 0.966556
Accuracy : 0.8894
Precision : 0.6863
Recall : 0.929712

Model Report Test
AUC Score : 0.894580
Accuracy : 0.8236
Precision : 0.5722
Recall : 0.828358
4244

Model Report Train
AUC Score : 0.967972
Accuracy : 0.8936
Precision : 0.6971
Recall : 0.926518

Model Report Test
AUC Score : 0.899022
Accuracy : 0.827
Precision : 0.5781
Recall : 0.828358
4254

Model Report Train
AUC Score : 0.968098
Accuracy : 0.8908
Precision : 0.6942
Recall : 0.913738

Model Report Test
AUC Score : 0.897983
Accuracy : 0.8136
Precision : 0.5567
Recall : 0.805970
4264

Model Report Train
AUC Score : 0.966973
Accuracy : 0.8929
Precision : 0.6954
Recall : 0.926518

Model Report Test
AUC Score : 0.898463
Accuracy : 0.822
Precision : 0.5714
Recall : 0.805970
4274

Model Report Train
AUC Score : 0.967452
Accuracy : 0.8951
Precision : 0.7005
Recall : 0.926518

Model Report Test
AUC Score : 0.897504
Accuracy : 0.8236
Precision : 0.5729
Recall : 0.820896
4284

Model Report Train
AU